## 잠깐만!
깃허브에 업로드하기 전에 ```API_KEY```를 지웠는지 체크하세요!

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
API_KEY = ""
CHANNEL_ID = ""

In [3]:
# 채널ID로 채널PlayList 찾기
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=API_KEY)
res = youtube.channels().list(id=CHANNEL_ID, part='contentDetails').execute()
CHANNEL_PLAYLIST = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

In [4]:
# 유튜브 데이터 추출하기 (PubDate, Title, ViewCount)
videos = youtube.playlistItems().list(playlistId=CHANNEL_PLAYLIST, part='snippet', maxResults=11).execute()
items = videos['items']
pubdate = []
title = []
viewcount = []

for item in items:
    pubdate.append(item['snippet']['publishedAt'])
    title.append(item['snippet']['title'])
    
    videoId = item['snippet']['resourceId']['videoId']
    view = youtube.videos().list(id=videoId, part='statistics').execute()
    viewcount.append(view['items'][0]['statistics']['viewCount'])

In [5]:
# 조회수 정렬하기
columns = ['pubdate','title','viewcount']
df = pd.DataFrame(data=zip(pubdate, title, viewcount), columns=columns)
df = df[1:]
df['viewcount'] = df['viewcount'].astype('int')
df = df.sort_values('viewcount', ascending=False)

In [38]:
df.head(3)

,pubdate,title,viewcount
7,2023-02-11T11:00:03Z,최근 항서제약이 주도하는 543명 진행성 간암 1차 치료제 캄렐리주맙+리보세라닙 임...,4587
5,2023-02-13T12:00:12Z,최근 완전관해가 발생한 따끈따끈한 리보세라닙+캄렐리주맙 임상2상 논문이 나왔습니다!,2775
9,2023-02-10T08:30:18Z,HLB가 인수한 피에스엠씨는 전환사채 발행이력이 깨끗한 기업입니다. 앞으로 많이 찍...,2204


## 최종코드

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from googleapiclient.discovery import build

def youtube_extract():

    API_KEY = ""
    CHANNEL_ID = ""

    # 채널ID로 채널PlayList 찾기
    youtube = build("youtube", "v3", developerKey=API_KEY)
    res = youtube.channels().list(id=CHANNEL_ID, part='contentDetails').execute()
    CHANNEL_PLAYLIST = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # 유튜브 데이터 추출하기 (PubDate, Title, ViewCount)
    videos = youtube.playlistItems().list(playlistId=CHANNEL_PLAYLIST, part='snippet', maxResults=11).execute()
    items = videos['items']
    pubdate = []
    title = []
    viewcount = []

    for item in items:
        pubdate.append(item['snippet']['publishedAt'])
        title.append(item['snippet']['title'])

        videoId = item['snippet']['resourceId']['videoId']
        view = youtube.videos().list(id=videoId, part='statistics').execute()
        viewcount.append(view['items'][0]['statistics']['viewCount'])

    # 조회수 정렬하기
    columns = ['pubdate','title','viewcount']
    df = pd.DataFrame(data=zip(pubdate, title, viewcount), columns=columns)
    df = df[1:]
    df['viewcount'] = df['viewcount'].astype('int')
    df = df.sort_values('viewcount', ascending=False)
    
    return df